In [1]:
# default_exp core

# module name here

> API details.

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.distributions as dist
import torch.nn as nn
import torch.nn.functional as F

In [5]:
from pp_lvm.simulations import get_time_from_intervals
from pp_lvm.models import encoder

In [2]:
torch.cuda.is_available()

True

# Constants

In [ ]:
INDUCING_POINT_STRIDE = 2

In [6]:
# Get Data

In [ ]:
data = {'I': I, 'Y': Y}
T_data = get_time_from_intervals(I)


In [ ]:
# Helpers

In [ ]:
#hide
from nbdev.showdoc import *